In [ ]:
import gc
import os

import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, classification_report

random_state = 42

In [ ]:
print(os.getcwd())

c:\Users\erich\Desktop\HYU\신용카드 고객 세그먼트 분류 공모전\eda


# A,B/C,D,E

In [ ]:
# train_set 준비

months = [7, 8, 9, 10, 11, 12]

trains = []

drop_columns = [
    'ID',
    'index',
    '기준년월',
    # '상품관련면제카드수_B0M',
    # '임직원면제카드수_B0M',
    # '우수회원면제카드수_B0M',
    # '기타면제카드수_B0M',
    '카드신청건수',
    '이용금액_R3M_체크',
    '이용금액_R3M_체크_가족',
    '유효카드수_체크',
    '유효카드수_체크_가족',
    '이용가능카드수_체크',
    '이용가능카드수_체크_가족',
    '이용카드수_체크',
    '이용카드수_체크_가족',
    '연회비발생카드수_B0M',
    '연회비할인카드수_B0M',
    '기본연회비_B0M',
    '제휴연회비_B0M',
    '할인금액_기본연회비_B0M',
    '할인금액_제휴연회비_B0M',
    '청구금액_기본연회비_B0M',
    '청구금액_제휴연회비_B0M',
    '캠페인접촉건수_R12M',
    '캠페인접촉일수_R12M'
]

for month in months:
    # train = pd.read_parquet(f"../data/train/1.회원정보/2018{month:02}_train_회원정보.parquet")
    # test = pd.read_parquet(f"../data/test/1.회원정보/2018{month:02}_test_회원정보.parquet")
    # customers.append(pd.concat([train, test], ignore_index=True))
    
    temp = pd.read_parquet(f"../data/train_preprocessed/train_preprocessed/2018{month:02}_train.parquet")
    temp["Segment"] = ((temp['Segment_A'] == 1) | (temp['Segment_B'] == 1)).astype(int)
    temp.drop([f"Segment_{a}" for a in "ABCDE"], axis=1, inplace=True)
    
    temp.drop(drop_columns, axis=1, inplace=True)

    trains.append(temp)
    
del temp
gc.collect()

0

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import xgboost as xgb

models = []
y_preds = []
idx = 7

importance_df = None  # 중요도 DataFrame 초기화

# 첫 번째 train으로 split 인덱스를 생성
first_train = trains[0]
X = first_train.drop("Segment", axis=1)
y = first_train["Segment"]
X_train_idx, X_test_idx = train_test_split(X.index, test_size=0.2, 
                                           random_state=42, stratify=y)

print("===== Macro F1 Score =====")

# 클래스 비율에 맞춰 scale_pos_weight 설정
# 다수 클래스: 399800, 소수 클래스: 200
scale_pos_weight = 399800 / 200  # 1999

# 모든 train에 대해 동일한 index로 나누기
for train in trains:
    train_X = train.drop("Segment", axis=1)
    train_y = train["Segment"]

    X_train = train_X.loc[X_train_idx]
    X_test = train_X.loc[X_test_idx]
    y_train = train_y.loc[X_train_idx]
    y_test = train_y.loc[X_test_idx]

    # XGBoost 모델 생성 (클래스 가중치 적용)
    model = xgb.XGBClassifier(
        objective='binary:logistic',
        eval_metric='logloss',
        use_label_encoder=False,
        random_state=42,
        scale_pos_weight=scale_pos_weight  # 클래스 가중치 추가
    )

    # 모델 학습
    model.fit(X_train, y_train)

    # 예측
    y_pred = model.predict(X_test)
    y_preds.append(y_pred)

    # 중요도 추출
    score_dict = model.get_booster().get_score(importance_type='gain')
    score_series = pd.Series(score_dict, name=f'model_{idx - 6}_importance')

    if importance_df is None:
        # 첫 모델이면 기준 feature 목록 생성
        features = model.get_booster().feature_names
        importance_df = pd.DataFrame({'feature': features})

    # Series → DataFrame으로 변환 후 merge
    score_df = score_series.reset_index()
    score_df.columns = ['feature', f'model_{idx - 6}_importance']
    
    importance_df = importance_df.merge(score_df, on='feature', how='left')

    models.append(model)

    # F1 점수 출력
    score = f1_score(y_test, y_pred, average='macro')
    print(f"{idx:02} : {round(score, 5)}")

    idx += 1

# NaN 값은 중요도 0으로 처리
importance_df.fillna(0, inplace=True)

# 중요도 총합 컬럼 추가 (선택)
importance_df['total_importance'] = importance_df.filter(like='model_').sum(axis=1)

# 정렬 (선택)
importance_df.sort_values(by='total_importance', ascending=False, inplace=True)


===== Macro F1 Score =====


c:\Users\erich\anaconda3\envs\CustomerSegment\lib\site-packages\xgboost\core.py:158: UserWarning: [02:23:53] WARNING: D:\bld\xgboost-split_1742444198260\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


07 : 0.67633


c:\Users\erich\anaconda3\envs\CustomerSegment\lib\site-packages\xgboost\core.py:158: UserWarning: [02:24:35] WARNING: D:\bld\xgboost-split_1742444198260\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


08 : 0.64272


...

10 : 0.60277


c:\Users\erich\anaconda3\envs\CustomerSegment\lib\site-packages\xgboost\core.py:158: UserWarning: [02:26:52] WARNING: D:\bld\xgboost-split_1742444198260\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


11 : 0.65053


c:\Users\erich\anaconda3\envs\CustomerSegment\lib\site-packages\xgboost\core.py:158: UserWarning: [02:27:37] WARNING: D:\bld\xgboost-split_1742444198260\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


12 : 0.63873


In [5]:
importance_df

,feature,model_1_importance,model_2_importance,model_3_importance,model_4_importance,model_5_importance,model_6_importance,total_importance
436,정상청구원금_B5M,18128.679688,18723.419922,48670.257812,12336.747070,54623.214844,2512.173828,154994.493164
428,정상청구원금_B0M,255.362366,396.400360,343.075928,1354.869385,264.476532,36854.507812,39468.692383
432,정상청구원금_B2M,996.360046,696.197144,213.007843,19137.132812,1455.144775,278.156433,22775.999054
671,홈페이지_금융건수_R6M,0.290104,0.490711,0.000000,3918.966309,0.000000,3.146142,3922.893266
502,잔액_리볼빙CA이월_B0M,0.000000,2460.760254,0.000000,584.792725,0.000000,0.000000,3045.552979
...,...,...,...,...,...,...,...,...
461,청구서발송여부_B0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
458,승인거절건수_입력오류_R3M,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
457,승인거절건수_BL_R3M,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
454,승인거절건수_기타_B0M,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
importance_df.to_csv(os.path.join(os.getcwd(), "xgb_ABvsCDE_feature_importance.csv"), index=False)

In [6]:
os.getcwd()

'c:\\Users\\erich\\Desktop\\HYU\\신용카드 고객 세그먼트 분류 공모전\\eda'